In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
# from statsmodels.tsa.stattools import adfuller, kpss
# from statsmodels.tsa.seasonal import seasonal_decompose
# from statsmodels.tsa.arima.model import ARIMA
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler
import warnings
from fbprophet import Prophet

%matplotlib inline
warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

In [2]:
df = pd.read_csv('data/all_features.csv')
df

,RegionID,time,for_sale_inv,mean_listing_price_cut,median_listing_price_cut,median_sale_price,perc_listing_with_price_cut,newly_pending_listings,mean_days_to_pending,median_days_to_pending,median_listing_price,home_value,concrete prices,plywood prices
0,102001,2018-01,1429548.0,NaN,NaN,240244.0,NaN,NaN,74.0,37.0,271600.0,227556.0,165.8,219.8
1,102001,2018-02,1386945.0,NaN,NaN,242084.0,NaN,NaN,73.0,33.0,272967.0,228816.0,166.1,227.9
2,102001,2018-03,1444213.0,13106.9871,6333.3333,243641.0,0.1177,NaN,67.0,26.0,279333.0,230080.0,168.9,236.5
3,102001,2018-04,1531525.0,13521.1971,7000.0000,244719.0,0.1220,NaN,59.0,19.0,288667.0,231351.0,169.0,236.6
4,102001,2018-05,1633549.0,13785.1816,8000.0000,245188.0,0.1341,NaN,53.0,18.0,295633.0,232608.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3802,753906,2020-10,7351.0,12010.7852,6391.6667,302819.0,0.1441,NaN,56.0,17.0,379711.0,292872.0,NaN,NaN
3803,753906,2020-11,7042.0,12060.9817,6000.0000,307058.0,0.1364,NaN,52.0,16.0,379900.0,295959.0,NaN,NaN
3804,753906,2020-12,6683.0,12650.3286,6683.3333,312299.0,0.1263,NaN,50.0,16.0,382933.0,299369.0,NaN,NaN
3805,753924,2018-03,3845.0,32847.6776,17833.3333,569239.0,0.0925,NaN,NaN,NaN,654667.0,715012.0,168.9,236.5


In [3]:
# top_1 = df[df.RegionID == 394399]
# df.groupby()

df_1 = df.groupby(by = ['RegionID']).agg({'mean_days_to_pending' : np.mean})

df_1.sort_values(by = 'mean_days_to_pending')[:10]

,mean_days_to_pending
RegionID,
395078,26.638889
395057,28.388889
395012,28.674419
394492,29.083333
394931,30.055556
394399,30.238095
395053,30.388889
394530,30.555556
395059,30.722222


In [4]:
df_1 = df.groupby(by = ['RegionID']).agg({'for_sale_inv' : sum})

df_1.sort_values(by = 'for_sale_inv')[:10]

,for_sale_inv
RegionID,
753924,7928.0
395134,61531.0
395115,74913.0
394952,76943.0
394931,82353.0
394666,87485.0
394308,89158.0
394843,89716.0
394619,90448.0


In [5]:
'''
ROI
Mean day to pending
For sale investory
'''

'\nROI\nMean day to pending\nFor sale investory\n'

In [6]:
from fbprophet import Prophet

# ! pip install fbprophet


def fbprophet_func(df, height=235000, train_size=.8, periods=13):
    '''
    The input dataframe must only have two columns called ds and y

    Returns the Original and Predictions along with the RMSE of the test data

    Will split the data into a train, test for you.

    Height is for the height of the arrow

    periods is for the periods you wish to predict
    '''
    df_prophet = df
    cutoff = round(df_prophet.shape[0] * train_size)
    train = df_prophet[:cutoff]
    test = df_prophet[cutoff:]

    model = Prophet()
    model.fit(train)
    forecast = model.predict(train)

    future = model.make_future_dataframe(periods=periods, freq='MS')
    future_forecast = model.predict(future)

    fig, ax = plt.subplots()

    sns.lineplot(original['time'], original['value'],
                 label='Original', color='r', linewidth=4)
    sns.lineplot(
        future_forecast['ds'], future_forecast['yhat'], label='Predictions', color='b')
    model.plot(future_forecast, ax=ax)
    plt.vlines(x=train['ds'].max(), ymin=future_forecast['yhat'].min(
    ) - 5, ymax=future_forecast['yhat'].max() + 5, linestyles='dashed')
    ax.set_title('Denver House Prices')
    ax.set_ylabel('House Prices')
    ax.set_xlabel('Time')
    plt.legend(loc='best')

    MSE = np.square(np.subtract(test.y, future_forecast.yhat)).mean()

    RMSE = math.sqrt(MSE)
    print(f'RMSE for the test data: {RMSE}')
    print("%RMSE: ", RMSE / original.mean())

In [7]:
df = pd.read_csv('data/zillow_data.csv')
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [8]:
def melt_data(df, compress = False):
    """
    Takes the zillow_data dataset in wide form or a subset of the zillow_dataset.  
    Returns a long-form datetime dataframe 
    with the datetime column names as the index and the values as the 'values' column.
    
    If more than one row is passes in the wide-form dataset, the values column
    will be the mean of the values from the datetime columns in all of the rows.
    
    If compress is True will compress data to only the value column.
    """
    if compress == True:
        melted = pd.melt(df, id_vars=['RegionName', 'RegionID', 'SizeRank', 'City', 'State', 
                                      'Metro', 'CountyName'], var_name='time')
        melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
        melted = melted.dropna(subset=['value'])
        return melted.groupby('time').aggregate({'value':'mean'})
    else:
        melted = pd.melt(df, id_vars=['RegionName', 'RegionID', 'SizeRank', 'City', 'State', 
                                      'Metro', 'CountyName'], var_name='time')
        melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
        melted = melted.dropna(subset=['value'])
        return melted

In [9]:
df = melt_data(df)
df.head()

,RegionName,RegionID,SizeRank,City,State,Metro,CountyName,time,value
0,60657,84654,1,Chicago,IL,Chicago,Cook,1996-04-01,334200.0
1,75070,90668,2,McKinney,TX,Dallas-Fort Worth,Collin,1996-04-01,235700.0
2,77494,91982,3,Katy,TX,Houston,Harris,1996-04-01,210400.0
3,60614,84616,4,Chicago,IL,Chicago,Cook,1996-04-01,498100.0
4,79936,93144,5,El Paso,TX,El Paso,El Paso,1996-04-01,77300.0


In [10]:
Denver_df = df[df['RegionName'] == 80216]

In [11]:
Denver_df

,RegionName,RegionID,SizeRank,City,State,Metro,CountyName,time,value
2966625,80216,93292,7303,Denver,CO,Denver,Denver,2013-01-01,91800.0
2981348,80216,93292,7303,Denver,CO,Denver,Denver,2013-02-01,93300.0
2996071,80216,93292,7303,Denver,CO,Denver,Denver,2013-03-01,95000.0
3010794,80216,93292,7303,Denver,CO,Denver,Denver,2013-04-01,97600.0
3025517,80216,93292,7303,Denver,CO,Denver,Denver,2013-05-01,100900.0
...,...,...,...,...,...,...,...,...,...
3835282,80216,93292,7303,Denver,CO,Denver,Denver,2017-12-01,251200.0
3850005,80216,93292,7303,Denver,CO,Denver,Denver,2018-01-01,253800.0
3864728,80216,93292,7303,Denver,CO,Denver,Denver,2018-02-01,258100.0
3879451,80216,93292,7303,Denver,CO,Denver,Denver,2018-03-01,263700.0


In [12]:
Denver_df_droped = Denver_df.drop(['RegionName', 'RegionID', 'SizeRank', 'City', 'Metro',
                              'CountyName', 'State'], axis=1)
cols = list(Denver_df_droped)[1:]

In [13]:
# cols

In [14]:
# train, test = Denver_df_droped[:int(
#     (len(Denver_df_droped) * .80))], Denver_df_droped[int(round(len(Denver_df_droped) * .80)):]

In [15]:
# train_float = train[cols].astype(float)
# test_float = test[cols].astype(float)
# scaler = StandardScaler()
# train_scaled = scaler.fit_transform(train_float)
# test_scaled = scaler.transform(test_float)

In [16]:
# sequence_length = 5
# window_length = sequence_length + 1

# x_train = []
# y_train = []
# for i in range(0, len(train) - window_length + 1):
#     window = train_scaled[i:i + window_length]
#     x_train.append(window[:-1, :])
#     y_train.append(window[-1, [-1]])
# x_train = np.array(x_train)
# y_train = np.array(y_train)

# x_test = []
# y_test = []
# for i in range(0, len(test) - window_length + 1):
#     window = test_scaled[i:i + window_length, :]
#     x_test.append(window[:-1, :])
#     y_test.append(window[-1, [-1]])
# x_test = np.array(x_test)
# y_test = np.array(y_test)

In [17]:
# model = Sequential()
# model.add(LSTM(64, activation = 'relu', input_shape = (5,1), return_sequences=True))
# model.add(LSTM(32, activation = 'relu', return_sequences=True))
# model.add(LSTM(16, activation = 'relu', return_sequences = False))
# model.add(Dropout(0.2))
# model.add(Dense(1))

# model.compile(optimizer='adam', loss = 'mse')
# model.summary()
# model.fit(x_train, y_train, epochs=1, validation_data=(x_test, y_test))

In [18]:
# y_predicts = model.predict(x_test)
# y_predict_inv_scaled = y_predicts*(train.value.max()-train.value.min())+train.value.min()
# y_predict = []
# for predict in y_predict_inv_scaled:
#     y_predict.append(predict[0])

# y_predict = pd.Series(y_predict, index=test.index[5:])


# y_test_inv_scaled = y_test*(train.value.max()-train.value.min())+train.value.min()
# y_test = []
# for predict in y_test_inv_scaled:
#     y_test.append(predict[0])
    
# y_test = pd.Series(y_test, index=test.index[5:])

In [19]:
# y_predict = pd.DataFrame({'time' : cols})

In [20]:
# fig, ax = plt.subplots(figsize=(15,10))
# plt.plot(y_test, color = 'blue', label='Actual Price')
# plt.plot(y_predict, color = 'orange',label='Predicted Price')
# plt.title('Denver House Prices', fontsize=16)
# plt.xlabel('Time')
# plt.ylabel('Home Value')
# plt.legend(loc='upper left', fontsize=8)
# # ax.set_xticklabels(test['time'])
# plt.show();

In [21]:
# y_predict = pd.DataFrame(y_predict, columns=Denver_df_droped.columns)
# y_predict

In [22]:
def LSTM_func(df, City=None, verbose=1, use_multiprocessing=False, epochs=5, batch_size=5, test_size=0.8):
    '''
    The dataframe can only have two columns, time and values drop everthing else before using this function
    you will break it!!

    Parameters
    -----------
    df = Dataframe (DON'T FORGET ONLY TIME AND VALUES)
    City = The name of the city you are modeling

    Returns
    ---------
    A plot
    '''
    cols = list(df)[1:]

    train, test = df[:int(
        (len(df) * .80))], df[int(round(len(df) * test_size)):]

    train_float = train[cols].astype(float)
    test_float = test[cols].astype(float)
    scaler = StandardScaler()
    train_scaled = scaler.fit_transform(train_float)
    test_scaled = scaler.transform(test_float)

    sequence_length = 5
    window_length = sequence_length + 1

    x_train = []
    y_train = []
    for i in range(0, len(train) - window_length + 1):
        window = train_scaled[i:i + window_length]
        x_train.append(window[:-1, :])
        y_train.append(window[-1, [-1]])
    x_train = np.array(x_train)
    y_train = np.array(y_train)

    x_test = []
    y_test = []
    for i in range(0, len(test) - window_length + 1):
        window = test_scaled[i:i + window_length, :]
        x_test.append(window[:-1, :])
        y_test.append(window[-1, [-1]])
    x_test = np.array(x_test)
    y_test = np.array(y_test)

    # Model building starts here
    model = Sequential()
    model.add(LSTM(64, activation='relu',
                   input_shape=(5, 1), return_sequences=True))
    model.add(LSTM(32, activation='relu', return_sequences=True))
    model.add(LSTM(16, activation='relu', return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mse')
    model.summary()
    # Fitting the model on the X_train and y_train
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(
        x_test, y_test), verbose=verbose, use_multiprocessing=use_multiprocessing)

    y_predicts = model.predict(x_train)
    y_predict_inv_scaled = scaler.inverse_transform(y_predicts)
    y_predict = []
    for predict in y_predict_inv_scaled:
        y_predict.append(predict[0])

    y_predict = pd.Series(y_predict, index=train.index[5:])

    y_test_inv_scaled = scaler.inverse_transform(y_train)
    y_train = []
    for predict in y_test_inv_scaled:
        y_train.append(predict[0])

    y_train = pd.Series(y_train, index=train.index[5:])
    
    MSE = np.square(np.subtract(y_train,y_predict)).mean() 
 
    RMSE = math.sqrt(MSE)
    print(f'RMSE: {RMSE}')
    print("%RMSE: ", RMSE/y_test.mean())

In [23]:
LSTM_func(Denver_df_droped, verbose=False, City = 'Denver', epochs=1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 5, 64)             16896     
_________________________________________________________________
lstm_1 (LSTM)                (None, 5, 32)             12416     
_________________________________________________________________
lstm_2 (LSTM)                (None, 16)                3136      
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 32,465
Trainable params: 32,465
Non-trainable params: 0
_________________________________________________________________
RMSE: 39635.23979245069
%RMSE:  17179.850697363938


In [24]:
# listing and turnover